In [ ]:
# PL: Split SQl Server ffrom Lakehouse

# System

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_lakehouse_sql_server - Start ----------\n")

In [ ]:
medallion_name = "Bronze"
if is_debug: print(f"medallion_name: {medallion_name}")

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

# Function

## Get "server_name" for "server_alias"

In [ ]:
# Used to apply "SQL Server aliases"
if "SQL Server" in list_technology:    
    def fn_get_server_name_for_server_alias(
        dict_server_alias_server_name
        , server_alias
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}
        
        try:
            server_name = None

            for key, value in dict_server_alias_server_name.items():
                if key == server_alias:
                    server_name = value
                    break

            alert             = "Success"
            alert_description = f"Server alias - server name: {server_alias} - {server_name}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, server_name)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Execute SQL query (on-premises SQL Server)

In [ ]:
# On-Premises SQL Server, not behind VPN
# PL: Will be reaplced with shortcuts
if "SQL Server" in list_technology:
    def fn_execute_sql_query(dict_parameter):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            username      = notebookutils.credentials.getSecret(dict_parameter["keyvault_url"], dict_parameter["keyvault_secret_name_user"])
            password      = notebookutils.credentials.getSecret(dict_parameter["keyvault_url"], dict_parameter["keyvault_secret_name_password"])
            server_name   = dict_parameter["server_name"]
            database_name = dict_parameter["database_name"]

            sdf = spark.read\
              .format("jdbc")\
              .option("url"                , f"jdbc:sqlserver://{server_name};databaseName={database_name}")\
              .option("query"              , dict_parameter["sql_query"])\
              .option("mssqlIsolationLevel", global_parameter.mssql_isolation_level)\
              .option("user"               , username)\
              .option("password"           , password)\
              .option("driver"             , "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
              .load()
            sdf_count = sdf.count()

            alert             = "Success"
            alert_description = f"Execute SQL Query (count): {sdf_count}"

            return (alert, alert_description, sdf, sdf_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, 0)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Add initial values in table "lh_cfg.extract_parameter"

In [ ]:
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_add_initial_extract_parameter(
        technology
        , frequency
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Edit this query to cover your "datetime_from_to" filters
            sql_code = f"""SELECT
    "{technology}"            AS `technology`
    , "{frequency}"           AS `frequency`
    , EO.`lakehouse_name`
    , EO.`server_name`
    , EO.`database_name`
    , EO.`schema_name`
    , EO.`table_name`
    , "datetime_from"           AS `name`
    , "1900-01-01 00:00:00.000" AS `value`
FROM
    (
              SELECT `technology`, `frequency`, `server_name`, `database_name`, `schema_name`, `table_name`, \"\" AS `lakehouse_name`, `sequence`, `prefix_select`, `column_name`, `from`, `where`, `is_pk`, `data_type`, `is_extracted`                 FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server`
        UNION SELECT `technology`, `frequency`, \"\" AS `server_name`, \"\" AS `database_name`, \"\" AS `schema_name`, `table_name`, `lakehouse_name`, `sequence`, `prefix_select`, `column_name`, `from`, `where`, `is_pk`, `data_type`, `is_extracted` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`
    )                                                                              AS EO
    LEFT JOIN delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter` AS EP
        ON EO.`technology`                  = EP.`technology`
        AND EO.`frequency`                  = EP.`frequency`
        AND IFNULL(EO.`lakehouse_name`, "") = IFNULL(EP.`lakehouse_name`, "")
        AND IFNULL(EO.`server_name`, "")    = IFNULL(EP.`server_name`, "")
        AND IFNULL(EO.`database_name`, "")  = IFNULL(EP.`database_name`, "")
        AND IFNULL(EO.`schema_name`, "")    = IFNULL(EP.`schema_name`, "")
        AND IFNULL(EO.`table_name`, "")     = IFNULL(EP.`table_name`, "")
WHERE
    EP.`technology`       IS NULL
    AND EO.`technology`   = \"{technology}\"
    AND EO.`frequency`    = \"{frequency}\"
    AND EO.`is_extracted` = 1
    AND EO.`where`        LIKE "%^datetime_from^%"

UNION ALL SELECT
    \"{technology}\"      AS `technology`
    , \"{frequency}\"     AS `frequency`
    , EO.`lakehouse_name`
    , EO.`server_name`
    , EO.`database_name`
    , EO.`schema_name`
    , EO.`table_name`
    , "datetime_to"       AS `name`
    , NULL                AS `value`
FROM
    (
              SELECT `technology`, `frequency`, `server_name`, `database_name`, `schema_name`, `table_name`, \"\" AS `lakehouse_name`, `sequence`, `prefix_select`, `column_name`, `from`, `where`, `is_pk`, `data_type`, `is_extracted`                 FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server`
        UNION SELECT `technology`, `frequency`, \"\" AS `server_name`, \"\" AS `database_name`, \"\" AS `schema_name`, `table_name`, `lakehouse_name`, `sequence`, `prefix_select`, `column_name`, `from`, `where`, `is_pk`, `data_type`, `is_extracted` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`
    )                                                                              AS EO
    LEFT JOIN delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter` AS EP
        ON EO.`technology`              = EP.`technology`
        AND EO.`frequency`                  = EP.`frequency`
        AND IFNULL(EO.`lakehouse_name`, "") = IFNULL(EP.`lakehouse_name`, "")
        AND IFNULL(EO.`server_name`, "")    = IFNULL(EP.`server_name`, "")
        AND IFNULL(EO.`database_name`, "")  = IFNULL(EP.`database_name`, "")
        AND IFNULL(EO.`schema_name`, "")    = IFNULL(EP.`schema_name`, "")
        AND IFNULL(EO.`table_name`, "")     = IFNULL(EP.`table_name`, "")
WHERE
    EP.`technology`       IS NULL
    AND EO.`technology`   = \"{technology}\"
    AND EO.`frequency`    = \"{frequency}\"
    AND EO.`is_extracted` = 1
    AND EO.`where`        LIKE "%^datetime_to^%"
ORDER BY
    `technology`
    , `frequency`
    , `lakehouse_name`
    , `server_name`
    , `database_name`
    , `schema_name`
    , `table_name`
    , `name`;"""

            rv_initial_extract_parameter = fn_execute_spark_sql(sql_code)
            sdf                          = rv_initial_extract_parameter[2]
            sdf_count                    = rv_initial_extract_parameter[3]

            if sdf_count > 0:
                # PL: 
                # Debug this - Start
                # Bug: Uses another sdf (the one from the "Test" section of fn_save_sdf_as_table())
                #fn_save_sdf_as_table("sdf", \
                #    "delta", \
                #    "append", \
                #    f"{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter", \
                #    None)

                sdf.write\
                    .format("delta")\
                    .mode("append")\
                    .save(f"{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter")
                # Debug this - End

            alert             = "Success"
            alert_description = f"New rows (count): {str(sdf_count)}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Update value "datetime_to" in table "extract_parameter"

In [ ]:
# Before the execution of the extraction loop:
#    • extraction_timeframe = Now --> datetime_to = now()
#    • extraction_timeframe IN (Previous Full Day,
#       Previous Full Week, Previous Full Month,
#       Previous Full Quarter, Previous Full Year) --> datetime_to = Today at 00:00:00.000
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_update_datetime_to_in_extract_parameter(technology, frequency):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        row        = None
        sdf_list   = None
        pdf_list   = None
        rv_update  = None
        sdf_update = None

        try:
            # Get all "extraction_timeframe"
            sql_code_extraction_timeframe = f"""SELECT DISTINCT `extraction_timeframe`
FROM
    (
              SELECT `technology`, `frequency`, `where`, `is_extracted`, `extraction_timeframe` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server`
        UNION SELECT `technology`, `frequency`, `where`, `is_extracted`, `extraction_timeframe` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`
    )
WHERE
    `technology`       = \"{technology}\"
    AND `frequency`    = \"{frequency}\"
    AND `where`        LIKE \"%datetime_to%\"
    AND `is_extracted` = 1;"""

            rv_extraction_timeframe        = fn_execute_spark_sql(sql_code_extraction_timeframe)
            sdf_extraction_timeframe       = rv_extraction_timeframe[2]
            sdf_extraction_timeframe_count = rv_extraction_timeframe[3]

            dict_count = {}
            count      = 0

            # If "extraction_timeframe" exists
            if sdf_extraction_timeframe_count > 0:
                sql_code_list = f"""SELECT CONCAT(
    `technology`, '|'
    , `frequency`, '|'
    , IFNULL(`lakehouse_name`, \"\"), '|'
    , IFNULL(`server_name`   , \"\"), '|'
    , IFNULL(`database_name` , \"\"), '|'
    , IFNULL(`schema_name`   , \"\"), '|'
    , IFNULL(`table_name`    , \"\")
) AS I
FROM
    (
              SELECT `technology`, `frequency`, '' AS `lakehouse_name`, `server_name`, `database_name`, `schema_name`, `table_name`, `where`, `extraction_timeframe`, `is_extracted`             FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server`
        UNION SELECT `technology`, `frequency`, `lakehouse_name`, '' AS `server_name`, '' AS `database_name`, '' AS `schema_name`, `table_name`, `where`, `extraction_timeframe`, `is_extracted` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`
    )
WHERE
    `technology`               = \"{technology}\"
    AND `frequency`            = \"{frequency}\"
    AND `where`                LIKE '%datetime_to%'
    AND `extraction_timeframe` = \"^extraction_timeframe^\"
    AND `is_extracted`         = 1;"""

                sql_code_update = f"""UPDATE delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter`
SET `value` = \"^value^\"
WHERE
    CONCAT(
        `technology`, '|',
        `frequency`, '|',
        IFNULL(`lakehouse_name`, \"\"), '|',
        IFNULL(`server_name`   , \"\"), '|',
        IFNULL(`database_name` , \"\"), '|',
        IFNULL(`schema_name`   , \"\"), '|',
        IFNULL(`table_name`    , \"\")
    ) IN (^list^)
    AND `name` LIKE \"%datetime_to%\"
    AND `frequency` = \"{frequency}\";"""

                # Loop "extraction_timeframe" for parameter "frequency"
                for row in sdf_extraction_timeframe.collect():
                    if is_debug: print(f"\n----- extraction_timeframe = \"{row.extraction_timeframe}\" - Start -----\n")
                    
                    sql_code_list  = sql_code_list.replace("^extraction_timeframe^", row.extraction_timeframe)
                    rv_list        = fn_execute_spark_sql(sql_code_list)
                    sdf_list       = rv_list[2]
                    sdf_list_count = rv_list[3]

                    if sdf_list_count > 0:
                        # PL: Define all the cases
                        if row.extraction_timeframe == "Now": value = fn_get_now("datetime")[1]
                        
                        if row.extraction_timeframe in ("Previous Full Day",
                            "Previous Full Day",
                            "Previous Full Week",
                            "Previous Full Month",
                            "Previous Full Quarter",
                            "Previous Full Year"):
                            value = f"{fn_get_now('date')[1]} 00:00:00.000"
                        
                        pdf_list = sdf_list.toPandas()
                        rv_list  = str(pdf_list['I'].tolist()).replace("[", "").replace("]", "")

                        sql_code_update = sql_code_update.replace("^value^", value)
                        sql_code_update = sql_code_update.replace("^list^", rv_list)

                        rv_update        = fn_execute_spark_sql(sql_code_update)
                        sdf_update       = rv_update[2]
                        sdf_update_count = rv_update[3]

                        sdf_update_count                     = sdf_update.collect()[0][0]
                        dict_count[row.extraction_timeframe] = sdf_update_count
                        count                                += sdf_update_count

                    if is_debug: print(f"\n----- extraction_timeframe = \"{row.extraction_timeframe}\" - End -----\n")

            alert             = "Success"
            alert_description = f"Updated rows (count): {str(count)}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, dict_count, count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Generate SELECT

In [ ]:
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_get_statament_select(
        sdf
        , technology
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            statament_select = f"SELECT "

            if technology == "SQL Server":
                for row in sdf.sort(sdf.sequence.asc()).collect():
                    statament_select += row.alias
            
            if technology == "Lakehouse":
                for row in sdf.sort(sdf.sequence.asc()).collect():
                    statament_select += f"`{row.column_name }`, "
                
                statament_select = statament_select[:-2]

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, statament_select)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Generate FROM

In [ ]:
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_get_statament_from(
        sdf
        , lakehouse_name
        , schema_name
        , table_name
        , technology
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            from_ = sdf.collect()[0]["from"]

            if from_ == "":
                if technology == "Lakehouse":
                    path = f"global_parameter.abfs_path_{lakehouse_name}"
                    statament_from = f"FROM delta.`{eval(path)}/Tables/{table_name}`"
                elif technology == "SQL Server":
                    statament_from = f"FROM [{schema_name}].[{table_name}]"
            else:
                if technology == "Lakehouse":
                    statament_from = f"FROM {from_}"
                    # replace "table_name" in "from_" with f"FROM delta.`{global_parameter.abfs_path_lh}_{lakehouse_name}/Tables/{table_name}`"
                elif technology == "SQL Server":
                    statament_from = f"FROM {from_}"

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, statament_from)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Generate WHERE

In [ ]:
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_get_statement_where(
        sdf
        , dict_parameter
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            statement_where = "WHERE "
            where_count     = sdf.filter(sf.col("where") != "").count()

            if where_count > 0:
                sql_code = f"""SELECT A.`where`
FROM
    (
        SELECT `technology`, `frequency`, '' AS `lakehouse_name`, `server_name`, `database_name`, `schema_name`, `table_name`, `where`, `is_extracted`                   FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server`
        UNION SELECT `technology`, `frequency`, `lakehouse_name`, '' AS `server_name`, '' AS `database_name`, '' AS `schema_name`, `table_name`, `where`, `is_extracted` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`
    ) AS A
WHERE
    A.`technology`                       = \"{dict_parameter["technology"]}\"
    AND A.`frequency`                    = \"{dict_parameter["frequency"]}\"
    AND IFNULL(A.`lakehouse_name`, \"\") = \"{dict_parameter["lakehouse_name"]}\"
    AND IFNULL(A.`server_name`   , \"\") = \"{dict_parameter["server_alias"]}\"
    AND IFNULL(A.`database_name` , \"\") = \"{dict_parameter["database_name"]}\"
    AND IFNULL(A.`schema_name`   , \"\") = \"{dict_parameter["schema_name"]}\"
    AND IFNULL(A.`table_name`    , \"\") = \"{dict_parameter["table_name"]}\"
    AND A.`where`                        IS NOT NULL
    AND A.`is_extracted`                 = 1;"""

                statement_where += fn_execute_spark_sql(sql_code)[2].collect()[0][0]

                sql_code = f"""SELECT
    `name`
    , `value`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter`
WHERE
    `technology`                       = \"{dict_parameter["technology"]}\"
    AND `frequency`                    = \"{dict_parameter["frequency"]}\"
    AND IFNULL(`lakehouse_name`, \"\") = \"{dict_parameter["lakehouse_name"]}\"
    AND IFNULL(`server_name`   , \"\") = \"{dict_parameter["server_alias"]}\"
    AND IFNULL(`database_name` , \"\") = \"{dict_parameter["database_name"]}\"
    AND IFNULL(`schema_name`   , \"\") = \"{dict_parameter["schema_name"]}\"
    AND IFNULL(`table_name`    , \"\") = \"{dict_parameter["table_name"]}\";"""

                sdf_extract_parameter = fn_execute_spark_sql(sql_code)[2]

                for row in sdf_extract_parameter.collect(): statement_where = statement_where.replace(f"^{row.name}^", f"'{row.value}'")

            if statement_where == "WHERE ": statement_where = ""

            alert             = "Success"
            alert_description = f""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, statement_where)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

In [ ]:
""" PL:
fn_append_to_lh_log_extract
    Add column "status" --> Use Logging Levels Debug, Info, Warn, Error, Fatal, instead of Bootstrap Success, Warning, Danger
    Add code to break "locals" to fn_name and caller_fn_name and get "Success" from all parent - child functions

fn_update_datetime_from_datetime_to_in_extract_parameter
    Add code to break "locals" to fn_name and caller_fn_name and get "Success" from all parent - child functions
    Example of failed extraction:
        fn_name                             caller_fn_name                          alert
        *******                             **************                          *****
        fn_extract_lakehouse_or_sql_server  ''                                      Success (parent)
        fn_save_in_single_file              fn_extract_lakehouse_or_sql_server      Danger (child)
    
Do we execute these 2 functions after "silver" (lh_bronze tables are recreated on every execution and we lose data if not in silver?
For example: in broze we have 10 tables, but these 10 tables end up to 5 tables in silver. Consider the entire silver "success" as "success"?
"""

## Append to table "lh_log.extract"

In [ ]:
# Append to table "lh_log.extract" the data for the successfully extracted tables
# PL: Create lakehouse "report" for things that need to be reported?
# PL: Skip this table as it can be generated from lh_log.log?
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_append_to_lh_log_extract():
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Create blank parameters, used to log "Danger"
            rv_sql_code_locals = None
            sdf_locals         = None
            pdf_list           = None
            rv_sdf             = None
            sdf                = None

            # Get the JSON column "locals_" for the successfully extracted table for the current "process_timestamp"
            sql_code_locals = f"""SELECT `locals`
FROM delta.`{global_parameter.abfs_path_lh_log}/Tables/log`
WHERE
  `medallion_name`        = \"Bronze\"
  AND `source_name`       IN(\"fn_extract_lakehouse_or_sql_server\", \"fn_extract_excel\", \"fn_extract_csv\", \"fn_extract_json\")
  AND `process_timestamp` = \"{global_parameter.process_timestamp}\" 
  AND `alert`             = \"Success\";"""
            rv_sql_code_locals = fn_execute_spark_sql(sql_code_locals)
            sdf_locals         = rv_sql_code_locals[2]
            sdf_locals_count   = rv_sql_code_locals[3]

            if sdf_locals_count > 0:
                # Read JSON and concatenate to pipe separated list
                sdf_locals = sdf_locals.withColumn("dict_parameter", sf.get_json_object(sf.col("locals"),"$.dict_parameter"))

                sdf_locals = sdf_locals.select(
                    sf.concat(
                        sf.get_json_object(sf.col("dict_parameter"),"$.technology")    .alias("technology")    , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"),"$.frequency")     .alias("frequency")     , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"),"$.lakehouse_name").alias("lakehouse_name"), sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"),"$.server_alias")  .alias("server_name")   , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"),"$.database_name") .alias("database_name") , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"),"$.schema_name")   .alias("schema_name")   , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"),"$.table_name")    .alias("table_name")
                    )
                    .alias("I")
                )

                pdf_list = sdf_locals.toPandas()
                rv_list  = str(pdf_list['I'].tolist()).replace("[", "").replace("]", "")

                sql_code = f"""SELECT
  T.`technology`
  , T.`frequency`
  , T.`lakehouse_name`
  , T.`server_name`
  , T.`database_name`
  , T.`schema_name`
  , T.`table_name`
  , T.`folder_name`
  , T.`file_name`
  , T.`worksheet_name`
  , MAX(CASE WHEN T.`A` = \"A\" THEN CAST(T.`value` AS TIMESTAMP) END) AS `extraction_date_start`
  , MAX(CASE WHEN T.`A` = \"B\" THEN CAST(T.`value` AS TIMESTAMP) END) AS `extraction_date_end`
  , \"{global_parameter.process_timestamp}\"                           AS `process_timestamp`
FROM
  (
    /* Get rows for date_from */
    SELECT
      \"A\" AS `A`
      , `technology`
      , `frequency`
      , `lakehouse_name`
      , `server_name`
      , `database_name`
      , `schema_name`
      , `table_name`
      , \"\" AS `folder_name`
      , \"\" AS `file_name`
      , \"\" AS `worksheet_name`
      , `value`
    FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter`
    WHERE
      CONCAT(
        `technology`                    , \"|\"
        , `frequency`                   , \"|\"
        , IFNULL(`lakehouse_name`, \"\"), \"|\"
        , IFNULL(`server_name`   , \"\"), \"|\"
        , IFNULL(`database_name` , \"\"), \"|\"
        , IFNULL(`schema_name`   , \"\"), \"|\"
        , IFNULL(`table_name`    , \"\")
      ) IN ({rv_list})
      AND `name` = \"datetime_from\"

    /* Get rows for date_to */
    UNION SELECT
      \"B\" AS `A`
      , `technology`
      , `frequency`
      , `lakehouse_name`
      , `server_name`
      , `database_name`
      , `schema_name`
      , `table_name`
      , \"\" AS `folder_name`
      , \"\" AS `file_name`
      , \"\" AS `worksheet_name`
      , `value`
    FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter`
    WHERE
      CONCAT(
        `technology`                    , \"|\"
        , `frequency`                   , \"|\"
        , IFNULL(`lakehouse_name`, \"\"), \"|\"
        , IFNULL(`server_name`   , \"\"), \"|\"
        , IFNULL(`database_name` , \"\"), \"|\"
        , IFNULL(`schema_name`   , \"\"), \"|\"
        , IFNULL(`table_name`    , \"\")
      ) IN ({rv_list})
      AND `name` = \"datetime_to\"

    /* Get rows for not filtered tables */
    UNION SELECT
      \"C\" AS A
      , C.`technology`
      , C.`frequency`
      , C.`lakehouse_name`
      , C.`server_name`
      , C.`database_name`
      , C.`schema_name`
      , C.`table_name`
      , C.`folder_name`
      , C.`file_name`
      , C.`worksheet_name`
      , NULL AS `value`
    FROM
      (
        SELECT
          `technology`
          , `frequency`
          , \"\" AS `lakehouse_name`
          , `server_name`
          , `database_name`
          , `schema_name`
          , `table_name`
          , \"\" AS `folder_name`
          , \"\" AS `file_name`
          , \"\" AS `worksheet_name`
          , `is_extracted`
        FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server`

        UNION SELECT
          `technology`
          , `frequency`
          , `lakehouse_name`
          , \"\" AS `server_name`
          , \"\" AS `database_name`
          , \"\" AS `schema_name`
          , `table_name`
          , \"\" AS `folder_name`
          , \"\" AS `file_name`
          , \"\" AS `worksheet_name`
          , `is_extracted`
        FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`

        UNION SELECT
          `technology`
          , `frequency`
          , \"\" AS `lakehouse_name`
          , \"\" AS `server_name`
          , \"\" AS `database_name`
          , \"\" AS `schema_name`
          , \"\" AS `table_name`
          , `folder_name`
          , `file_name`
          , `worksheet_name`
          , `is_extracted`
        FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_excel`

        UNION SELECT
          `technology`
          , `frequency`
          , \"\" AS `lakehouse_name`
          , \"\" AS `server_name`
          , \"\" AS `database_name`
          , \"\" AS `schema_name`
          , \"\" AS `table_name`
          , `folder_name`
          , `file_name`
          , \"\" AS `worksheet_name`
          , `is_extracted`
        FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_csv`

        UNION SELECT
          `technology`
          , `frequency`
          , \"\" AS `lakehouse_name`
          , \"\" AS `server_name`
          , \"\" AS `database_name`
          , \"\" AS `schema_name`
          , \"\" AS `table_name`
          , `folder_name`
          , `file_name`
          , \"\" AS `worksheet_name`
          , `is_extracted`
        FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_json`
      ) AS C
    WHERE
      CONCAT(
        C.`technology`                    , \"|\"
        , C.`frequency`                   , \"|\"
        , IFNULL(C.`lakehouse_name`, \"\"), \"|\"
        , IFNULL(C.`server_name`   , \"\"), \"|\"
        , IFNULL(C.`database_name` , \"\"), \"|\"
        , IFNULL(C.`schema_name`   , \"\"), \"|\"
        , IFNULL(C.`table_name`    , \"\")
      ) IN ({rv_list})
      AND C.`is_extracted` = 1
  ) AS T
GROUP BY
  T.`technology`
  , T.`frequency`
  , T.`lakehouse_name`
  , T.`server_name`
  , T.`database_name`
  , T.`schema_name`
  , T.`table_name`
  , T.`folder_name`
  , T.`file_name`
  , T.`worksheet_name`
ORDER BY
  T.`technology`
  , T.`frequency`
  , T.`lakehouse_name`
  , T.`server_name`
  , T.`database_name`
  , T.`schema_name`
  , T.`table_name`
  , T.`folder_name`
  , T.`file_name`
  , T.`worksheet_name`;"""
                rv_sdf    = fn_execute_spark_sql(sql_code)
                sdf       = rv_sdf[2]
                sdf_count = rv_sdf[3]

                sdf.write.format("delta")\
                    .mode("append")\
                    .save(f"{global_parameter.abfs_path_lh_log}/Tables/extract")

            if "sdf_count" not in locals(): sdf_count = 0

            alert             = "Success"
            alert_description = f"Inserted rows (count): {sdf_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description)
        finally:
          if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par

          fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Update values 'datetime_from' and "datetime_to" in table "lh_cfg.extract_parameter"

In [ ]:
# Update (after the execution of the extraction loop):
#   • "datetime_from" with "datatime_to"
#   • "datetime_to" with NULL
# for the successfully extracted tables
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_update_datetime_from_datetime_to_in_extract_parameter(pdf_log):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Get the JSON column "locals_" for the successfully extracted tables
            pdf_locals = pdf_log.loc[
                (pdf_log["medallion_name"] == "Bronze")
                & (pdf_log["source_name"] == "fn_extract_lakehouse_or_sql_server")
                & (pdf_log["alert"] == "Success")
            , ["locals"]]
            pdf_locals_count = len(pdf_locals.index)        

            if pdf_locals_count > 0:
                # Create spark dataframe
                sdf_locals = spark.createDataFrame(pdf_locals)
                sdf_locals_count = sdf_locals.count()

                # Read JSON and concatenate to "pipe separated list"
                sdf_locals = sdf_locals.withColumn("dict_parameter", sf.get_json_object(sf.col("locals"),"$.dict_parameter"))
                sdf_locals = sdf_locals.select(
                    sf.concat(
                        sf.get_json_object(sf.col("dict_parameter"), "$.technology")    .alias("technology")    , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"), "$.frequency")     .alias("frequency")     , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"), "$.lakehouse_name").alias("lakehouse_name"), sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"), "$.server_alias")  .alias("server_name")   , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"), "$.database_name") .alias("database_name") , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"), "$.schema_name")   .alias("schema_name")   , sf.lit("|"),
                        sf.get_json_object(sf.col("dict_parameter"), "$.table_name")    .alias("table_name")
                    )
                    .alias("I")
                )

                pdf_list = sdf_locals.toPandas()
                rv_list  = str(pdf_list['I'].tolist()).replace("[", "").replace("]", "")

                sql_code = f"""SELECT
    `technology`
    , `frequency`
    , `lakehouse_name`
    , `server_name`
    , `database_name`
    , `schema_name`
    , `table_name`
    , `value`
    , CAST(NULL AS VARCHAR(8000)) AS `N`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter`
WHERE
    CONCAT(
        `technology`              , \"|\"
        , `frequency`             , \"|\"
        , IFNULL(`lakehouse_name`, \"\"), \"|\"
        , IFNULL(`server_name`   , \"\"), \"|\"
        , IFNULL(`database_name` , \"\"), \"|\"
        , IFNULL(`schema_name`   , \"\"), \"|\"
        , IFNULL(`table_name`    , \"\")
    ) IN ({rv_list})
    AND name = \"datetime_to\";"""
                sdf_src = fn_execute_spark_sql(sql_code)[2]
                
                # Get the destination table (lh_cfg.extract_parameter)
                dstn = dlt.DeltaTable.forPath(spark, f"{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter")

                # Update "datetime_from" with "datatime_to" for the successfully extracted tables - End #
                dstn.alias("dstn") \
                .merge(
                    sdf_src.alias("src"),
                    "coalesce(dstn.technology       , \"\") = coalesce(src.technology    , \"\") \
                    and coalesce(dstn.frequency     , \"\") = coalesce(src.frequency     , \"\") \
                    and coalesce(dstn.lakehouse_name, \"\") = coalesce(src.lakehouse_name, \"\") \
                    and coalesce(dstn.server_name   , \"\") = coalesce(src.server_name   , \"\") \
                    and coalesce(dstn.database_name , \"\") = coalesce(src.database_name , \"\") \
                    and coalesce(dstn.schema_name   , \"\") = coalesce(src.schema_name   , \"\") \
                    and coalesce(dstn.table_name    , \"\") = coalesce(src.table_name    , \"\") \
                    and dstn.name                           = 'datetime_from'"
                ) \
                .whenMatchedUpdate(set = {"value": "src.value"}) \
                .execute()
                # Update "datetime_from" with "datatime_to" for the successfully extracted tables - End#

                # Update "datetime_to" with NULL for the successfully extracted tables - Start # 
                dstn.alias("dstn") \
                .merge(
                    sdf_src.alias("src"),
                    "coalesce(dstn.technology       , \"\") = coalesce(src.technology    , \"\") \
                    and coalesce(dstn.frequency     , \"\") = coalesce(src.frequency     , \"\") \
                    and coalesce(dstn.lakehouse_name, \"\") = coalesce(src.lakehouse_name, \"\") \
                    and coalesce(dstn.server_name   , \"\") = coalesce(src.server_name   , \"\") \
                    and coalesce(dstn.database_name , \"\") = coalesce(src.database_name , \"\") \
                    and coalesce(dstn.schema_name   , \"\") = coalesce(src.schema_name   , \"\") \
                    and coalesce(dstn.table_name    , \"\") = coalesce(src.table_name    , \"\") \
                    and dstn.name                           = 'datetime_to'"
                ) \
                .whenMatchedUpdate(set = {"value": "src.N"}) \
                .execute()
                # Update "datetime_to" with NULL for the successfully extracted tables - End #                
         
            alert             = "Success"
            alert_description = ""

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"
            
            return (alert, alert_description)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Operation

## Extract technology "Lakehouse", "SQL Server"

In [ ]:
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_extract_lakehouse_or_sql_server(technology, frequency, dict_server_alias_server_name):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        #if is_debug: par = {}

        try:
            now_year  = fn_get_now("year")[1]
            now_month = fn_get_now("month")[1]

            # Add initial parameters in table "lh_cfg.extract_parameter"
            fn_add_initial_extract_parameter(technology, frequency)

            # Add now() as datetime_to in table "lh_cfg.extract_parameter" for all the tables
            fn_update_datetime_to_in_extract_parameter(technology, frequency)

            # Get extract table list
            dict_parameter_table_list     = {"technology": technology, "frequency": frequency, "action": "Extract"}
            rv_extract_table_list         = fn_get_extract_table_list(dict_parameter_table_list)
            sdf_extract_table_list_status = rv_extract_table_list[0]
            sdf_extract_table_list        = rv_extract_table_list[2]
            sdf_extract_table_list_count  = rv_extract_table_list[3]
            if is_debug:
                print(f"dict_parameter_table_list: {dict_parameter_table_list}")
                print(f"rv_extract_table_list: {rv_extract_table_list}")
                print(f"sdf_extract_table_list_status: {sdf_extract_table_list_status}")
                print(f"sdf_extract_table_list:")
                display(sdf_extract_table_list)
                print(f"sdf_extract_table_list_count:  {sdf_extract_table_list_count}")

            # Loop extract table list
            if sdf_extract_table_list_count > 0:
                # etl_r --> sdf_extract_table_list.row
                for etl_r in sdf_extract_table_list \
                    .sort(sdf_extract_table_list.technology.asc(),
                        sdf_extract_table_list.frequency.asc(),
                        sdf_extract_table_list.lakehouse_name.asc(),
                        sdf_extract_table_list.server_name.asc(),
                        sdf_extract_table_list.database_name.asc(),
                        sdf_extract_table_list.schema_name.asc(),
                        sdf_extract_table_list.table_name.asc()
                    ).collect():

                    if technology == "SQL Server":
                        server_alias = etl_r.server_name
                        server_name  = fn_get_server_name_for_server_alias(dict_server_alias_server_name, server_alias)[2]

                    if technology == "Lakehouse":
                        server_alias = etl_r.server_name
                        server_name  = etl_r.server_name
                    
                    if is_debug:
                        print(f"server_alias: {type(server_alias)} | {server_alias}")
                        print(f"server_name: {type(server_name)} | {server_name}")                        
                        print(f"sdf_extract_table_list.etl_r:")  
                        print(f"  technology:                    {etl_r.technology}")
                        print(f"  frequency:                     {etl_r.frequency}")
                        print(f"  lakehouse_name:                {etl_r.lakehouse_name}")
                        print(f"  server_name:                   {etl_r.server_name}")
                        print(f"  database_name:                 {etl_r.database_name}")
                        print(f"  schema_name:                   {etl_r.schema_name}")
                        print(f"  table_name:                    {etl_r.table_name}")
                        print(f"  keyvault_url:                  {etl_r.keyvault_url}")
                        print(f"  keyvault_secret_name_user:     {etl_r.keyvault_secret_name_user}")
                        print(f"  keyvault_secret_name_password: {etl_r.keyvault_secret_name_password}")

                    dict_parameter = {"process_timestamp": global_parameter.process_timestamp
, "technology": etl_r.technology
, "frequency": etl_r.frequency
, "lakehouse_name": etl_r.lakehouse_name
, "server_alias": server_alias
, "server_name": server_name
, "database_name": etl_r.database_name
, "schema_name": etl_r.schema_name
, "table_name": etl_r.table_name
, "keyvault_url": etl_r.keyvault_url
, "keyvault_secret_name_user": etl_r.keyvault_secret_name_user
, "keyvault_secret_name_password": etl_r.keyvault_secret_name_password}

                    if is_debug: print(f"dict_parameter: {type(dict_parameter)} | {dict_parameter}")

                    # Generate column list
                    rv_extract_column         = fn_get_extract_column(dict_parameter)
                    sdf_extract_column_status = rv_extract_column[0]
                    sdf_extract_column        = rv_extract_column[2]
                    sdf_extract_column_count  = rv_extract_column[3]
                    if is_debug:
                        print(f"sdf_extract_column_status: {sdf_extract_column_status}")
                        print(f"sdf_extract_column:")
                        display(sdf_extract_column)
                        print(f"sdf_extract_column_count: {sdf_extract_column_count}")

                    # Generate dynamic SELECT statement
                    rv_stmt_select     = fn_get_statament_select(sdf_extract_column, etl_r.technology)
                    stmt_select_status = rv_stmt_select[0]
                    stmt_select        = rv_stmt_select[2]
                    if is_debug:
                        print(f"stmt_select_status: {stmt_select_status}")
                        print(f"stmt_select:        {stmt_select}")

                    # Generate dynamic FROM statement
                    rv_stmt_from     = fn_get_statament_from(sdf_extract_column, etl_r.lakehouse_name, etl_r.schema_name, etl_r.table_name, technology)
                    stmt_from_status = rv_stmt_from[0]
                    stmt_from        = rv_stmt_from[2]
                    if is_debug:
                        print(f"stmt_from_status: {stmt_from_status}")
                        print(f"stmt_from: {stmt_from}")

                    # Generate dynamic WHERE statement
                    rv_stmt_where     = fn_get_statement_where(sdf_extract_column, dict_parameter)
                    stmt_where_status = rv_stmt_where[0]
                    stmt_where        = rv_stmt_where[2]
                    if technology == "Lakehouse":
                        stmt_where = stmt_where.replace("[", "`").replace("]", "`")
                    if is_debug:
                        print(f"stmt_where_status: {stmt_where_status}")
                        print(f"stmt_where: {stmt_where}")

                    # Generate SELECT FROM WHERE statement
                    stmt_extract = f"{stmt_select} {stmt_from} {stmt_where}"
                    dict_parameter["sql_query"] = stmt_extract
                    if is_debug:
                        print(f"stmt_extract: {stmt_extract}")
                        print(f"dict_parameter: {dict_parameter}")

                    # Extract to SDF - Start
                    # On-premises SQL Server
                    if technology == "SQL Server":
                        if is_debug: print("Extract from on-premises SQL Server")
                        rv_execute_sql_query = fn_execute_sql_query(dict_parameter)

                    # Lakehouse
                    if technology == "Lakehouse":
                        if is_debug: 
                            print("technology = \"Lakehouse\"")
                            print(f"dict_parameter: {dict_parameter}")
                        rv_execute_sql_query = fn_execute_spark_sql(stmt_extract)
                    
                    sdf_extracted_status = rv_execute_sql_query[0]
                    sdf_extracted        = rv_execute_sql_query[2]
                    sdf_extracted_count  = rv_execute_sql_query[3]

                    if is_debug:
                        print(f"rv_execute_sql_query: {rv_execute_sql_query}")
                        print(f"sdf_extracted_status: {sdf_extracted_status}")
                        print(f"sdf_extracted (schema):")
                        sdf_extracted.printSchema()
                        print(f"sdf_extracted:")
                        display(sdf_extracted)
                        print(f"sdf_extracted_count: {sdf_extracted_count}")
                    # Extract to SDF - End

                    if sdf_extracted_status == "Success":
                        # Save extracted dataframe as table in "lh_bronze" - Start
                        lakehouse_name_clean = fn_clean_accent_in_string(etl_r.lakehouse_name)[2]
                        server_name_clean    = fn_clean_accent_in_string(etl_r.server_name)[2]
                        database_name_clean  = fn_clean_accent_in_string(etl_r.database_name)[2]
                        schema_name_clean    = fn_clean_accent_in_string(etl_r.schema_name)[2]
                        table_name_clean     = fn_clean_accent_in_string(etl_r.table_name)[2]

                        lakehouse_name_clean = fn_clean_not_alphanumeric_in_string(lakehouse_name_clean, "_")[2]
                        server_name_clean    = fn_clean_not_alphanumeric_in_string(server_name_clean, "_")[2]
                        database_name_clean  = fn_clean_not_alphanumeric_in_string(database_name_clean, "_")[2]
                        schema_name_clean    = fn_clean_not_alphanumeric_in_string(schema_name_clean, "_")[2]
                        table_name_clean     = fn_clean_not_alphanumeric_in_string(table_name_clean, "_")[2]

                        dict_parameter["lakehouse_name_clean"] = lakehouse_name_clean
                        dict_parameter["server_name_clean"] = server_name_clean
                        dict_parameter["database_name_clean"] = database_name_clean
                        dict_parameter["schema_name_clean"] = schema_name_clean
                        dict_parameter["table_name_clean"] = table_name_clean

                        if technology == "SQL Server": sdf_extracted_table_path = f"{global_parameter.abfs_path_lh_bronze}/Tables/{server_name_clean}\
_{database_name_clean}_{schema_name_clean}_{table_name_clean}"

                        if technology == "Lakehouse": sdf_extracted_table_path = f"{global_parameter.abfs_path_lh_bronze}/Tables/{lakehouse_name_clean}\
_{table_name_clean}"
                        fn_save_sdf_as_table(
                            sdf_extracted
                            , "delta"
                            , "overwrite"
                            , sdf_extracted_table_path
                            , dict_parameter
                        )

                        if is_debug:
                            print("sdf_extracted - Start")
                            print(f"sdf_extracted_table_path: {sdf_extracted_table_path}")
                            sdf_extracted.printSchema()
                            display(sdf_extracted)
                            print("sdf_extracted - End")
                        # Save extracted dataframe as table in "lh_bronze" - End

                        # Save extracted dataframe as single .parquet file in "lh_bronze/Processed/<technology>" (archive) - Start
                        if technology == "SQL Server": sdf_extracted_archive_path = f"{global_parameter.abfs_path_lh_bronze}/Files/processed/{technology}/\
{server_name_clean}/{database_name_clean}/{schema_name_clean}/{table_name_clean}/\
{now_year}/{now_month}/\
{table_name_clean}_{global_parameter.process_timestamp}.parquet"

                        if technology == "Lakehouse": sdf_extracted_archive_path = f"{global_parameter.abfs_path_lh_bronze}/Files/processed/{technology}/\
{lakehouse_name_clean}/{table_name_clean}/\
{now_year}/{now_month}/\
{table_name_clean}_{global_parameter.process_timestamp}.parquet"

                        rv_save_in_single_file = fn_save_in_single_file(sdf_extracted, sdf_extracted_archive_path)
                        if is_debug:
                            print(f"alert: {rv_save_in_single_file[0]}")
                            print(f"alert_description: {rv_save_in_single_file[1]}")
                        # Save extracted dataframe as single .parquet file in "lh_bronze/Processed/<technology>" (archive) - End

                    # Insert "Success" in lh_log (for each table) - Start
                    alert             = "Success"
                    alert_description = f"Row count: {str(sdf_extracted_count)}"

                    if is_debug:
                        par = {}
                        par["locals"] = locals()
                        fn_print_debug_info(alert, fn_name, par)
                        del par
                    
                    fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)
                    # Insert "Success" in lh_log (for each table) - End

                    if is_debug: print(f"\n------------------------------ Loop - End ------------------------------\n")

            return (alert, alert_description)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            if is_debug:
                par = {}
                par["locals"] = locals()
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

            return (alert, alert_description)

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_lakehouse_sql_server - End ----------\n")